In [3]:
'''
    Lagarange_equation_RR 
    anthor : anthony Create on  2022 10 01
'''

import sympy
import math
import numpy as np
from Inertia_tensor_symbol import compute_dh_matrix,Inertia_tensor
sympy.init_printing(use_latex='mathjax')
from IPython.display import display

In [4]:
#RR arm test
#create some symbols discription 
n = 2
pi = sympy.pi
q = sympy.symbols(f'q1:{n+1}')#input angle
qd = sympy.symbols(f'qd1:{n+1}')#input angle vel
qdd = sympy.symbols(f'qdd1:{n+1}')#input angle acc
mass = sympy.symbols(f'm1:{n+1}')
g = sympy.Symbol('g')
I1 = sympy.Symbol('I1')
I2 = sympy.Symbol('I2')
I = [I1,I2]

In [5]:
#create Transform matrix discription
l1 = sympy.Symbol('l1')
l2 = sympy.Symbol('l2')
S = sympy.sin
C = sympy.cos

T1 = sympy.MutableDenseMatrix([[C(q[0]),-S(q[0]),0,0],
                      [S(q[0]),C(q[0]),0,0],
                      [0,0,1,0],
                      [0,0,0,1]])
T2 = sympy.MutableDenseMatrix([[C(q[1]),-S(q[1]),0,l1],
                      [S(q[1]),C(q[1]),0,0],
                      [0,0,1,0],
                      [0,0,0,1]])

T3 = sympy.MutableDenseMatrix([[1,0,0,l2],
                                [0,1,0,0],
                                [0,0,1,0],
                                [0,0,0,1]])

Ti = []
Ti.append(T1 @ T2)
Ti.append(T1 @ T2 @ T3)

display(Ti)

⎡⎡-sin(q₁)⋅sin(q₂) + cos(q₁)⋅cos(q₂)  -sin(q₁)⋅cos(q₂) - sin(q₂)⋅cos(q₁)  0  l
⎢⎢                                                                            
⎢⎢sin(q₁)⋅cos(q₂) + sin(q₂)⋅cos(q₁)   -sin(q₁)⋅sin(q₂) + cos(q₁)⋅cos(q₂)  0  l
⎢⎢                                                                            
⎢⎢                0                                   0                   1   
⎢⎢                                                                            
⎣⎣                0                                   0                   0   

₁⋅cos(q₁)⎤  ⎡-sin(q₁)⋅sin(q₂) + cos(q₁)⋅cos(q₂)  -sin(q₁)⋅cos(q₂) - sin(q₂)⋅co
         ⎥  ⎢                                                                 
₁⋅sin(q₁)⎥  ⎢sin(q₁)⋅cos(q₂) + sin(q₂)⋅cos(q₁)   -sin(q₁)⋅sin(q₂) + cos(q₁)⋅co
         ⎥, ⎢                                                                 
   0     ⎥  ⎢                0                                   0            
         ⎥  ⎢                                      

In [8]:
#angular velocity jacobian (Jw)
Jw = sympy.MutableDenseNDimArray(np.zeros((n,3,n),dtype=np.float64))
for i in range(n):
    jw = sympy.MutableDenseNDimArray([0,0,1]).reshape(3,1)
    for j in range(0,i):
        #RR always ratate Zaxis
        jw = np.hstack((jw,Ti[j][0:3,2]))
    jw = np.hstack((jw,np.zeros((3,n-i))))
    Jw[i,:,:] = jw

display(Jw)

⎡⎡ 0   0⎤  ⎡0  0⎤⎤
⎢⎢      ⎥  ⎢    ⎥⎥
⎢⎢ 0   0⎥  ⎢0  0⎥⎥
⎢⎢      ⎥  ⎢    ⎥⎥
⎣⎣1.0  0⎦  ⎣1  1⎦⎦

In [5]:
#linear velocity jocobian  (Jv) 
cx = sympy.symbols(f'cx1:{n+1}')
cy = sympy.symbols(f'cy1:{n+1}')
cz = sympy.symbols(f'cz1:{n+1}')
P = sympy.MutableDenseMatrix(np.eye(4,dtype=np.float64))

Jv = sympy.MutableDenseNDimArray(np.zeros((n,3,n),dtype=np.float64))

for i in range(n):
    ''' 
        in this case Assume joint mass center on their transformer matrix position
        if you center are not in this try below to clculate you center position on each joint
        P = Ti[i][:,:] @ sympy.MutableDenseMatrix([[1,0,0,cx[i]],
                                                [0,1,0,cy[i]],
                                                [0,0,1,cz[i]],
                                                [0,0,0,1]])

    '''

    P = Ti[i][:,:] 
    x = P[0,3]
    y = P[1,3]
    z = P[2,3]
    for j in range(n):  
        Jv[i,:,j] =  sympy.Array([sympy.diff(x,q[j]).simplify(),sympy.diff(y,q[j]).simplify(),sympy.diff(z,q[j]).simplify()])

    display(Jv)

⎡⎡-l₁⋅sin(q₁)  0⎤  ⎡0  0⎤⎤
⎢⎢              ⎥  ⎢    ⎥⎥
⎢⎢l₁⋅cos(q₁)   0⎥  ⎢0  0⎥⎥
⎢⎢              ⎥  ⎢    ⎥⎥
⎣⎣     0       0⎦  ⎣0  0⎦⎦

⎡⎡-l₁⋅sin(q₁)  0⎤  ⎡-l₁⋅sin(q₁) - l₂⋅sin(q₁ + q₂)  -l₂⋅sin(q₁ + q₂)⎤⎤
⎢⎢              ⎥  ⎢                                               ⎥⎥
⎢⎢l₁⋅cos(q₁)   0⎥  ⎢l₁⋅cos(q₁) + l₂⋅cos(q₁ + q₂)   l₂⋅cos(q₁ + q₂) ⎥⎥
⎢⎢              ⎥  ⎢                                               ⎥⎥
⎣⎣     0       0⎦  ⎣              0                       0        ⎦⎦

In [6]:
#Petential energy and Kentic energy and Mass martix M
P = sympy.MutableDenseMatrix(np.eye(4,dtype=np.float64))
PE = 0
#M is Mass Matrix M  Ki is each joint Kentic energy
M = sympy.MutableDenseMatrix(np.zeros((n,n),dtype = np.float64))
Ki = []

'''
    in this case Assume not have Rotation Kentic energy
    so Ki = mass[i] * Jv[i]' * Jv[i]
    have Rotation Kentic energy the formula can be :
    Ki = mass[i] * Jv[i]' * Jv[i] + I[i] * Jw[i]' * Jw[i]
    Mass_Matrix = Sum(Ki)
    KE = 0.5 * (qd' * Mass_Matrix * qd ) 
    in this case g is Assume on Yaxis g = [0,-9.8,0] 
    PE = sum(mass[i]* g * h)
'''

for i in range(n):
    display( sympy.MutableDenseMatrix(I[i] * np.dot(Jw[i].transpose() ,Jw[i])))
    M = M + (mass[i] * sympy.MutableDenseMatrix(np.dot(Jv[i].transpose(),Jv[i])))
    Ki.append( mass[i] * sympy.MutableDenseMatrix(np.dot(Jv[i].transpose(),Jv[i])))

    P = Ti[i][:,:] 
    PE = PE +  g * mass[i]* P[1,3] 

KE = (0.5 * sympy.MutableDenseMatrix(qd).transpose() @ M @ sympy.MutableDenseMatrix(qd))
display(PE)
display(Ki)
display(KE)

⎡1.0⋅I₁  0⎤
⎢         ⎥
⎣  0     0⎦

⎡I₂  I₂⎤
⎢      ⎥
⎣I₂  I₂⎦

g⋅l₁⋅m₁⋅sin(q₁) + g⋅m₂⋅(l₁⋅sin(q₁) + l₂⋅(sin(q₁)⋅cos(q₂) + sin(q₂)⋅cos(q₁)))

⎡⎡   ⎛  2    2         2    2    ⎞   ⎤  ⎡                  ⎛                  
⎢⎢m₁⋅⎝l₁ ⋅sin (q₁) + l₁ ⋅cos (q₁)⎠  0⎥  ⎢               m₂⋅⎝(-l₁⋅sin(q₁) - l₂⋅
⎢⎢                                   ⎥, ⎢                                     
⎢⎣               0                  0⎦  ⎢                                     
⎣                                       ⎣m₂⋅(-l₂⋅(-l₁⋅sin(q₁) - l₂⋅sin(q₁ + q₂

             2                                 2⎞                             
sin(q₁ + q₂))  + (l₁⋅cos(q₁) + l₂⋅cos(q₁ + q₂)) ⎠                  m₂⋅(-l₂⋅(-l
                                                                              
                                                                              
))⋅sin(q₁ + q₂) + l₂⋅(l₁⋅cos(q₁) + l₂⋅cos(q₁ + q₂))⋅cos(q₁ + q₂))             

                                                                              
₁⋅sin(q₁) - l₂⋅sin(q₁ + q₂))⋅sin(q₁ + q₂) + l₂⋅(l₁⋅cos(q₁) + l₂⋅cos(q₁ + q₂))⋅
                                                  

⎡    ⎛                                                                        
⎣qd₁⋅⎝0.5⋅m₂⋅qd₂⋅(-l₂⋅(-l₁⋅sin(q₁) - l₂⋅sin(q₁ + q₂))⋅sin(q₁ + q₂) + l₂⋅(l₁⋅co

                                                 ⎛   ⎛  2    2         2    2 
s(q₁) + l₂⋅cos(q₁ + q₂))⋅cos(q₁ + q₂)) + 0.5⋅qd₁⋅⎝m₁⋅⎝l₁ ⋅sin (q₁) + l₁ ⋅cos (

   ⎞      ⎛                               2                                 2⎞
q₁)⎠ + m₂⋅⎝(-l₁⋅sin(q₁) - l₂⋅sin(q₁ + q₂))  + (l₁⋅cos(q₁) + l₂⋅cos(q₁ + q₂)) ⎠

⎞⎞       ⎛                                                                    
⎠⎠ + qd₂⋅⎝0.5⋅m₂⋅qd₁⋅(-l₂⋅(-l₁⋅sin(q₁) - l₂⋅sin(q₁ + q₂))⋅sin(q₁ + q₂) + l₂⋅(l

                                                        ⎛  2    2             
₁⋅cos(q₁) + l₂⋅cos(q₁ + q₂))⋅cos(q₁ + q₂)) + 0.5⋅m₂⋅qd₂⋅⎝l₂ ⋅sin (q₁ + q₂) + l

 2    2         ⎞⎞⎤
₂ ⋅cos (q₁ + q₂)⎠⎠⎦

In [7]:
#gravity Matrix G(q)
'''
    gravity Matrix G(q) = diff(PE)/diff(q[i]) is n * 1 matrix
'''
G = sympy.MutableDenseMatrix(np.zeros((n,1),dtype=np.float64))
for i in range(n):
    G[i] = sympy.diff(PE,q[i]).simplify()

display(G)

⎡g⋅(l₁⋅m₁⋅cos(q₁) + m₂⋅(l₁⋅cos(q₁) + l₂⋅cos(q₁ + q₂)))⎤
⎢                                                     ⎥
⎣                g⋅l₂⋅m₂⋅cos(q₁ + q₂)                 ⎦

In [8]:
#Christoffel symbol Matrix 
c = sympy.MutableDenseNDimArray(np.zeros((n,n,n),dtype=np.float64))
for k in range(n):
    for i in range(n):
        for j in range(n):
            c[i,j,k] = 0.5 * (sympy.diff(M[k,j],q[i]) + sympy.diff(M[k,i],q[j]) - sympy.diff(M[i,j],q[k]))


C = sympy.MutableDenseMatrix(np.zeros((n,n),dtype=np.float64))
for k in range(n):
    for j in range(n):
        temp = 0
        for i in range(n):
            temp = temp + c[i,j,k] * qd[i]
        C[k,j] = temp.simplify()

#this is C(q,qd) Matrix is n * n matrix
display(C)
#this is V(q,qd) Matrix is n * 1 matrix
display(sympy.MutableDenseMatrix(np.dot(C,qd)))

⎡ -l₁⋅l₂⋅m₂⋅qd₂⋅sin(q₂)    -l₁⋅l₂⋅m₂⋅(qd₁ + 1.0⋅qd₂)⋅sin(q₂)⎤
⎢                                                           ⎥
⎣1.0⋅l₁⋅l₂⋅m₂⋅qd₁⋅sin(q₂)                  0                ⎦

⎡-l₁⋅l₂⋅m₂⋅qd₁⋅qd₂⋅sin(q₂) - l₁⋅l₂⋅m₂⋅qd₂⋅(qd₁ + 1.0⋅qd₂)⋅sin(q₂)⎤
⎢                                                                ⎥
⎢                                   2                            ⎥
⎣                   1.0⋅l₁⋅l₂⋅m₂⋅qd₁ ⋅sin(q₂)                    ⎦